# The Spinning Effective One-Body Hamiltonian

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body Hamiltonian as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.

### Throughout this module, we will refer to [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010 and [Barausse and Buonanno (2011)](https://arxiv.org/abs/1107.2904) as BB2011.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows:

0. [Introduction](#intro): The Physical System of Interest
1. [Step 1](#Hreal): The Real Hamiltonian $H_{\rm real}$
1. [Step 2](#Heff): The Effective Hamiltonian $H_{\rm eff}$
1. [Step 3](#Heff_terms): Terms of $H_{\rm eff}$  
    a. [Step 3a](#Hs): Leading Order Spin Effects $H_{\rm S}$  
    b. [Step 3b](#Hns): The Nonspinning Hamiltonian $H_{\rm NS}$  
    c. [Step 3c](#Hd): The Quadrupole Deformation $H_{\rm D}$
1. [Step 4](#Hso): The Spin-Orbit Term $H_{\rm SO}$  
    a. [Step 4a](#HsoTerm1): $H_{\rm SO}$ Term 1  
    b. [Step 4b](#HsoTerm2coeff): $H_{\rm SO}$ Term 2 Coefficient  
    c. [Step 4c](#HsoTerm2): $H_{\rm SO}$ Term 2  
    d. [Step 4ci](#HsoTerm2a): $H_{\rm SO}$ Term 2a  
    e. [Step 4cii](#HsoTerm2b): $H_{\rm SO}$ Term 2b  
    f. [Step 4ciii](#HsoTerm2c): $H_{\rm SO}$ Term 2c
1. [Step 5](#Hss): The Spin-Spin Term $H_{\rm SS}$  
    a. [Step 5a](#HssTerm1): $H_{\rm SS}$ Term 1  
    b. [Step 5b](#HssTerm2coeff): $H_{\rm SS}$ Term 2 coefficient  
    c. [Step 5c](#HssTerm2): $H_{\rm SS}$ Term 2  
    d. [Step 5d](#HssTerm3coeff): $H_{\rm SS}$ Term 3 coefficient  
    e. [Step 5e](#HssTerm3): $H_{\rm SS}$ Term 3  
[Step NOT](#HsoHsscommon): Expressions Common to $H_{\rm SO}$ and $H_{\rm SS}$
1. [Step 6](#HnsTerms): The $H_{\rm NS}$ Terms
1. [Step 7](#HdTerms): The $H_{\rm D}$ Terms  
    a.[Step 8a](#Hdcoeff): $H_{\rm D}$ Coefficient  
    b.[Step 8b](#Hdsum): $H_{\rm D}$ Sum
1. [Step N-5](#metpots): The Deformed and Rescaled Metric Potentials
1. [Step N-4](#metpotderivs): Derivatives of the Metric Potential
1. [Step U](#dotproducts): Dot Products
1. [Step N-3](#Hreal_spin_combos): $H_{\rm real}$ Spin Combinations ${\bf S}^{*}$ and ${\bf S}_{\rm Kerr}$
1. [Step N-2](#Heff_spin_combos): $H_{\rm eff}$ Spin Combinations $\boldsymbol{\sigma}$ and $\boldsymbol{\sigma}^{*}$
1. [Step N-1](#mass_terms): Mass Terms $\mu$, $M$, and $\eta$
1. [Step N](#metric): The Metric $g$ of Choice
1. [Notes](#notes)
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two black holes with masses $m_{1}$, $m_{2}$ and spins ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian [$H_{\rm real}$](#Hreal) describes the dynamics of this system; we will define [$H_{\rm real}$](#Hreal) as in BB2010 Section VE.  There, [$H_{\rm real}$](#Hreal) is canonically transformed and mapped to an effective Hamiltonian [$H_{\rm eff}$](#Heff) describing the motion of a test particle of mass [$\mu$](#mass_terms) and spin [${\bf S}_{\rm Kerr}$](#Hreal_spin_combos).  Here we seek to break up [$H_{\rm real}$](#Hreal) and document the terms in such a way that the resulting Python code can be used to numerically evaluate [$H_{\rm real}$](#Hreal).  

We will use Greek letters to denote spacetime tensor indicies, and spacial tensor indicies with lowercase Latin letters.

<font color='red'>Please note: LALSuite computes $H_{\rm real}$ up to a constant multiple of $M$ (*I think*).</font>

<a id='Hreal'></a>

# Step 1: The real Hamiltonian $H_{\textrm{real}}$ \[Back to [top](#toc)\]
$$\label{Hreal}$$

The SEOB Hamiltonian $H_{\rm real}$ is given by Equation (5.69) of BB2010:

\begin{equation}
    H_{\rm real} = M \sqrt{ 1 + 2 \eta \left( \frac{ H_{\rm eff} }{ \mu } - 1 \right) }.
\end{equation}

Here [$H_{\rm eff}$](#Heff) is an [*effective* Hamiltonian](#intro) and [$M$](#mass_terms), [$\mu$](#mass_terms), and [$\eta$](#mass_terms) are constants determined by $m_{1}$ and $m_{2}$.

<font color='red'>The code in LALSimIMRSpinEOBHamiltonianPrec.c claims the origin of $H_{\textrm{real}}$ is Equation (2) of [Taracchini, Pan, Buonanno, et al](https://arxiv.org/abs/1202.0790):

\begin{equation}
    H_{\rm real} = M \sqrt{ 1 + 2 \nu \left( \frac{ H_{\rm eff} }{ \mu } - 1 \right) } - M
\end{equation}

where $\nu = \eta = \frac{\mu}{M}$.  Double-check this!</font>

In [1]:
%%writefile SEOBNR/Hamiltonian.txt
Hreal = M * sp.sqrt(1 + 2*eta*(Heff/mu - 1))

Overwriting SEOBNR/Hamiltonian.txt


<a id='Heff'></a>

# Step 2: The Effective Hamiltonian $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{Heff}$$

The effective Hamiltonian $H_{\rm eff}$ is given by Equation (5.70) of BB2010.  We combine the notation of BB2010 Equations (5.70) and (5.43) with our notation [$H_{\rm D}$](#Hd) to write

\begin{equation}
    H_{\rm eff} = H_{\rm S} + H_{\rm NS} \underbrace{ -\frac{ \eta }{ 2 r^{3} } \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j} }_{H_{\rm D}}.
\end{equation}

Here [$H_{\rm S}$](#Hs) denotes leading order effects of spin-spin and spin-orbit coupling, [$H_{\rm NS}$](#Hns) is the Hamiltonian for a nonspinning test particle, and [$H_{\rm D}$](#Hd) describes quadrupole deformation of the coupling of the particle's spin with itself to leading order.

In [2]:
%%writefile -a SEOBNR/Hamiltonian.txt

Heff = Hs + Hns + Hd

Appending to SEOBNR/Hamiltonian.txt


<a id='Heff_terms'></a>

# Step 3: Terms of $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{Heff_terms}$$

In this step, we break down each of the terms [$H_{\rm S}$](#Hs), [$H_{\rm NS}$](#Hns), and [$H_{\rm D}$](#Hd) in [$H_{\rm eff}$](#Heff).

<a id='Hs'></a>

## Step 3a: Leading Order Spin Effects $H_{\rm S}$ \[Back to [top](#toc)\]
$$\label{Hs}$$

From BB2010 Equation (4.17),

\begin{equation}
    H_{\rm S} = H_{\rm SO} + H_{\rm SS}
\end{equation}

where [$H_{\rm SO}$](#Hso) includes spin-orbit terms and [$H_{\rm SS}$](#Hss) includes spin-spin terms.

In [3]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hs = Hso + Hss

Appending to SEOBNR/Hamiltonian.txt


<a id='Hns'></a>

## Step 3b: The Nonspinning Hamiltonian $H_{\rm NS}$ \[Back to [top](#toc)\]
$$\label{Hns}$$

From BB2010 Equation (5.43),

\begin{equation}
    H_{\rm NS} = \underbrace{ \beta^{i} p_{i} }_{\beta p\ {\rm sum}} + \alpha \sqrt{ \smash[b]{\underbrace{ \mu^{2} + \gamma^{ij} p_{i} p_{j} + {\cal Q}_{4} \left( p \right) }_{H_{\rm NS}\ {\rm radicand}}}}
\end{equation}

We define [$\beta p\ {\rm sum}$](#betapsum), [$\alpha$](#alpha), and [$H_{\rm NS}\ {\rm radicand}$](#Hnsradicand) elsewhere.

In [4]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hns = betapsum + alpha*sp.sqrt(Hnsradicand)

Appending to SEOBNR/Hamiltonian.txt


<a id='Hd'></a>

## Step 3c: The Quadrupole Deformation $H_{\rm D}$ \[Back to [top](#toc)\]
$$\label{Hd}$$

We break up $H_{\rm D}$ as follows:

\begin{equation}
    H_{\rm D} = \underbrace{ -\frac{ \eta }{ 2 r^{3} } }_{H_{\rm D}\ {\rm coefficient}} \underbrace{ \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j} }_{H_{\rm D}\ {\rm sum}}
\end{equation}

For further dissection, see [$H_{\rm D}\ {\rm coefficient}$](#Hdcoeff) and [$H_{\rm D}\ {\rm sum}$](#Hdsum).

In [5]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hd = Hdcoeff*Hdsum

Appending to SEOBNR/Hamiltonian.txt


<a id='Hso'></a>

# Step 4: The Spin-Orbit Term $H_{\rm SO}$ \[Back to [top](#toc)\]
$$\label{Hso}$$

We will write BB2010 Equation (4.18) as:

\begin{align}
    H_{\rm SO} = H_{\rm SO}\ {\rm Term\ 1} + H_{\rm SO}\ {\rm Term\ 2\ coefficient} * H_{\rm SO}\ {\rm Term\ 2}.
\end{align}

We define [$H_{\rm SO}\ {\rm Term\ 1}$](#HsoTerm1), [$H_{\rm SO}\ {\rm Term\ 2\ coefficient}$](#HsoTerm2coeff), and [$H_{\rm SO}\ {\rm Term\ 2}$](#HsoTerm2) elsewhere.

In [6]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hso = HsoTerm1 + HsoTerm2coeff*HsoTerm2

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm1'></a>
## Step 4a: $H_{\rm SO}$ Term 1 \[Back to [top](#toc)\]
$$\label{HsoTerm1}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 1} = \frac{ e^{2 \nu - \tilde{\mu} } \left( e^{\tilde{\mu} + \nu} - \tilde{B} \right) \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( {\bf S} \cdot \hat{\bf S}_{\rm Kerr} \right) }{ \tilde{B}^{2} \sqrt{Q} \xi^{2} }
\end{equation}

We define [$e^{2\nu}$](#exp2nu), [$e^{\tilde{\mu}}$](#exp2mu), [$e^{\nu}$](#exp2nu), [$\tilde{B}$](#Btilde), [$\hat{\bf p} \cdot \boldsymbol{\xi} r$](#pdotxir), and [${\bf S} \cdot \hat{\bf S}_{\rm Kerr}$](#SdotSkerrhat) elsewhere.

In [7]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm1 = exp2nu*(expmu*expnu - Btilde)*pdotxir*SdotSkerr/(expmu*Btilde**2*sp.sqrt(Q)*xisq)

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2coeff'></a>
## Step 4b: $H_{\rm SO}$ Term 2 Coefficient \[Back to [top](#toc)\]
$$\label{HsoTerm2coeff}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2\ coefficient} = \frac{ e^{\nu - 2 \tilde{\mu}} }{ \tilde{B}^{2} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2} }
\end{equation}

We define [$e^{\nu}$](#exp2nu), [$e^{2\tilde{\mu}}$](#exp2mu), [$\tilde{B}$](#Btilde), [$Q$](#Q), and [$\xi^{2}$](#xi) elsewhere.

In [12]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm2coeff = expnu/(exp2mu*Btilde**2*(sp.sqrt(Q) + 1)*sp.sqrt(Q)*xi2)

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2'></a>
## Step 4c: $H_{\rm SO}$ Term 2 \[Back to [top](#toc)\]
$$\label{HsoTerm2}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2} = \underbrace{ \left( {\bf S} \cdot \boldsymbol{\xi} \right) \tilde{J} \left[ \mu_r \left( \hat{\bf p} \cdot {\bf v} r \right) \left( \sqrt{Q} + 1 \right) - \mu_{\cos \theta} \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} -\sqrt{Q} \left( \nu_r \left( \hat{\bf p} \cdot {\bf v} r \right) + \left( \mu_{\cos \theta} - \nu_{\cos \theta} \right) \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} \right) \right] \tilde{B}^{2} }_{H_{\rm SO}\ {\rm Term\ 2a}} + \underbrace{ e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( 2 \sqrt{Q} + 1 \right) \left[ \tilde{J} \nu_r \left( {\bf S} \cdot {\bf v} \right) - \nu_{\cos \theta} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B} }_{H_{\rm SO}\ {\rm Term\ 2b}} - \underbrace{ \tilde{J} \tilde{B}_{r} e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( \sqrt{Q} + 1 \right) \left( {\bf S} \cdot {\bf v} \right) }_{H_{\rm SO}\ {\rm Term\ 2c}}
\end{equation}

In [13]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm2 = HsoTerm2a + HsoTerm2c - HsoTerm2c

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2a'></a>
### $H_{\rm SO}$ Term 2a \[Back to [top](#toc)\]
$$\label{HsoTerm2a}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2a} = \left( {\bf S} \cdot \boldsymbol{\xi} \right) \tilde{J} \left[ \mu_r \left( \hat{\bf p} \cdot {\bf v} r \right) \left( \sqrt{Q} + 1 \right) - \mu_{\cos \theta} \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} -\sqrt{Q} \left( \nu_r \left( \hat{\bf p} \cdot {\bf v} r \right) + \left( \mu_{\cos \theta} - \nu_{\cos \theta} \right) \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} \right) \right] \tilde{B}^{2}
\end{equation}

We define [${\bf S} \cdot \boldsymbol{\xi}$](#Sdotxi), [$\tilde{J}$](#Jtilde), [$\mu_{r}$](#mur), [$\hat{\bf p} \cdot {\bf v} r$](#pdotvr), [$Q$](#Q), [$\mu_{\cos \theta}$](#mucostheta), [$\hat{\bf p} \cdot {\bf n}$](#pdotn), [$\xi^{2}$](#xi), [$\nu_{r}$](#nur), [$\nu_{\cos\theta}$](#nucostheta), and [$\tilde{B}$](#Btilde) elsewhere.

In [14]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm2a = Sdotxi*Jtilde*(mur*pdotvr*(sp.sqrt(Q)+1)
                           - mucostheta*pdotn*xisq
                           - sp.sqrt(Q)*(nur*pdotnr + (mucostheta - nucostheta)*pdotn*xisq))*Btilde**2

Appending to SEOBNR/Hamiltonian.txt


<a id='nur'></a>
### $\nu_{r}$ \[Back to [top](#toc)\]
$$\label{nur}$$

From BB2010 Equation (5.47c) we have

\begin{equation}
    \nu_{r} =  \frac{ r }{ \Sigma } + \frac{ \varpi^{2} \left( \varpi^{2} \Delta^{\prime}_{t} - 4 r \Delta_{t} \right) }{ 2 \Lambda_{t} \Delta_{t} }.
\end{equation}

We define [$\Sigma$](#Sigma), [$\varpi^{2}$](#w2), [$\Delta_{t}$](#Deltat), and [$\Lambda_{t}$](#Lambdat) elsewhere.

In [19]:
%%writefile -a SEOBNR/Hamiltonian.txt

nur = r/Sigma + w2*(w2*Deltatprm - 4*r*Deltat)/(2*Lambdat*Deltat)

Appending to SEOBNR/Hamiltonian.txt


<a id='nucostheta'></a>
### $\nu_{\cos\theta}$ \[Back to [top](#toc)\]
$$\label{nucostheta}$$

From BB2010 Equation (5.47g) we have

\begin{equation}
    \nu_{\cos\theta} = \frac{ a^{2} \varpi^{2} \cos\theta \left( \varpi^{2} - \Delta_{t} \right) }{ \Lambda_{t} \Sigma }.
\end{equation}

We define [$a$](#a_terms), [$\varpi^{2}$](#w2), [$\cos\theta$](#costheta), [$\Delta_{t}$](#Deltat), [$\Lambda_{t}$](#Lambdat), and [$\Sigma$](#Sigma) elsewhere.

In [20]:
%%writefile -a SEOBNR/Hamiltonian.txt

nucostheta = a2*w2*costheta*(w2 - Deltat)/(Lambdat*Sigma)

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2b'></a>
### $H_{\rm SO}$ Term 2b \[Back to [top](#toc)\]
$$\label{HsoTerm2b}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2b} = e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( 2 \sqrt{Q} + 1 \right) \left[ \tilde{J} \nu_r \left( {\bf S} \cdot {\bf v} \right) - \nu_{\cos \theta} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B}
\end{equation}

We define [$e^{\tilde{\mu}}$](#exp2mu), [$e^{\tilde{\nu}}$](#exp2nu), [${\bf p} \cdot \xi r$](#pdotxir), [$Q$](#Q), [$\tilde{J}$](#Jtilde), [$\nu_{r}$](#nur), [${\bf S} \cdot {\bf v}$](#Sdotv), [$\nu_{\cos\theta}$](#nucostheta), [${\bf S} \cdot {\bf n}$](#Sdotn), [$\xi^{2}$](#xi), and [$\tilde{B}$](#Btilde) elsewhere.

In [21]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm2b = expmu*expnu*pdotxir*(2*sp.sqrt(Q)+1)*(Jtilde*nur*Sdotv - nucostheta*Sdotn*xisq)*Btilde

Appending to SEOBNR/Hamiltonian.txt


<a id='Sdotv'></a>
### ${\bf S} \cdot {\bf v}$ \[Back to [top](#toc)\]
$$\label{Sdotv}$$

We have

\begin{equation}
    {\bf S} \cdot {\bf v} = S^{1} v^{1} + S^{2} v^{2} + S^{3} v^{3}.
\end{equation}

We define [${\bf v}$](#v) elsewhere.

In [85]:
%%writefile -a SEOBNR/Hamiltonian.txt

Sdotv = S1*v1 + S2*v2 + S3*v3

Appending to SEOBNR/Hamiltonian.txt


<a id='Sdotn'></a>
### ${\bf S} \cdot {\bf n}$ \[Back to [top](#toc)\]
$$\label{Sdotn}$$

We have

\begin{equation}
    {\bf S} \cdot {\bf n} = S^{1} n^{1} + S^{2} n^{2} + S^{3} n^{3}.
\end{equation}

We define [${\bf n}$](#n) elsewhere.

In [86]:
%%writefile -a SEOBNR/Hamiltonian.txt

Sdotn = S1*n1 + S2*n2 + S3*n3

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm2c'></a>
### $H_{\rm SO}$ Term 2c \[Back to [top](#toc)\]
$$\label{HsoTerm2c}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2c} = \tilde{J} \tilde{B}_{r} e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( \sqrt{Q} + 1 \right) \left( {\bf S} \cdot {\bf v} \right)
\end{equation}

We define [$\tilde{J}$](#Jtilde), [$\tilde{B}_{r}$](#Brtilde), [$e^{\tilde{\mu}}$](#exp2mu), [$e^{\nu}$](#exp2nu), [${\bf p} \cdot \xi r$](#pdotxir), [$Q$](#Q), and [${\bf S} \cdot {\bf v}$](#Sdotv) elsewhere.

In [22]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm2c = Jtilde*Brtilde*expmu*expnu*pdotxir*(sp.sqrt(Q)+1)*Sdotv

Appending to SEOBNR/Hamiltonian.txt


<a id='Brtilde'></a>

## $\tilde{B}_{r}$ \[Back to [top](#toc)\]
$$\label{Brtilde}$$

From BB2010 Equation (5.49), we have

\begin{equation}
    \tilde{B}_{r} = \frac{ \sqrt{ \Delta_{r} } \Delta_{t}^{\prime} - 2 \Delta_{t} }{ 2 \sqrt{ \Delta_{r} \Delta_{t} } }.
\end{equation}

We define [$\Delta_{r}$](#Deltar), [$\Delta_{t}^{\prime}$](#Deltatprm), and [$\Delta_{t}$](#Deltat) elsewhere.

In [87]:
%%writefile -a SEOBNR/Hamiltonian.txt

Brtilde = (sp.sqrt(Deltar)*Deltatprm - 2*Deltat)/(2*sp.sqrt(Deltar*Deltat))

Appending to SEOBNR/Hamiltonian.txt


<a id='Hss'></a>

# Step 5: The Spin-Spin Term $H_{\rm SS}$ \[Back to [top](#toc)\]
$$\label{Hss}$$

We will write BB2010 Equation (4.19) as

\begin{equation}
    H_{\rm SS} = H_{\rm SS}\ {\rm Term\ 1} + H_{\rm SS}\ {\rm Term\ 2\ coefficient} \left( H_{\rm SS}\ {\rm Term\ 2} \right) + H_{\rm SS}\ {\rm Term\ 3\ coefficient} \left( H_{\rm SS}\ {\rm Term\ 3} \right).
\end{equation}

We define [$H_{\rm SS}$ Term 1](#HssTerm1), [$H_{\rm SS}$ Term 2 coefficient](#HssTerm2coeff), [$H_{\rm SS}$ Term 2](#HssTerm2), [$H_{\rm SS}$ Term 3 coefficient](#HssTerm3coeff), and [$H_{\rm SS}$ Term 3](#HssTerm3) elsewhere.

<a id='HssTerm1'></a>
## Step 5a: $H_{\rm SS}$ Term 1 \[Back to [top](#toc)\]
$$\label{HssTerm1}$$

Combining BB2010 Equation (4.19) with [this](#Hss), we have

\begin{equation}
    H_{\rm SS}\ {\rm Term\ 1} = \omega \left( {\bf S} \cdot \hat{\bf S}_{\rm Kerr} \right).
\end{equation}

We define [$\omega$](#w) and [${\bf S} \cdot \hat{\bf S}_{\rm Kerr}$](#SdotSkerrhat) elsewhere.

In [88]:
%%writefile -a SEOBNR/Hamiltonian.txt

HssTerm1 = omega*SdotSkerrhat

Appending to SEOBNR/Hamiltonian.txt


<a id='omega'></a>
### $\omega$ \[Back to [top](#toc)\]
$$\label{omega}$$

From BB2010 Equation (5.31) we have

\begin{equation}
    \omega = \frac{ \tilde{\omega}_{\rm fd} }{ \Lambda_{t} }.
\end{equation}

We define [$\tilde{\omega}_{\rm fd}$](#omegatilde) and [$\Lambda_{t}$](#Lambdat) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

omega = omegatilde/Lambdat

<a id='SdotSkerrhat'></a>
### ${\bf S} \cdot \hat{\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{SdotSkerrhat}$$

We have

\begin{equation}
    {\bf S} \cdot \hat{\bf S}_{\rm Kerr} = S^{1} \hat{\bf S}_{\rm Kerr}^{1} + S^{2} \hat{\bf S}_{\rm Kerr}^{2} + S^{3} \hat{\bf S}_{\rm Kerr}^{3}.
\end{equation}

We define [$\hat{\bf S}_{\rm Kerr}$](#Skerrhat) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

SdotSkerrhat = S1*Skerrhat1 + S2*Skerrhat2 + S3*Skerrhat3

<a id='Skerrhat'></a>
### $\hat{\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{Skerrhat}$$

From BB2010 Equation (4.24), we have

\begin{equation}
    \hat{\bf S}_{\rm Kerr} = \frac{ {\bf S}_{\rm Kerr} }{ \left\lvert {\bf S}_{\rm Kerr} \right\rvert } = \frac{ {\bf S}_{\rm Kerr}}{ \sqrt{ {\bf S}_{\rm Kerr}^{1} {\bf S}_{\rm Kerr}^{1} + {\bf S}_{\rm Kerr}^{2} {\bf S}_{\rm Kerr}^{2} + {\bf S}_{\rm Kerr}^{3} {\bf S}_{\rm Kerr}^{3} } }.
\end{equation}

We define [${\bf S}_{\rm Kerr}$](#Skerr) elsewhere.

In [89]:
%%writefile -a SEOBNR/Hamiltonian.txt

Skerrhat1 = S1/sp.sqrt(Skerr1*Skerr1 + Skerr2*Skerr2 + Skerr3*Skerr3)
Skerrhat2 = S2/sp.sqrt(Skerr1*Skerr1 + Skerr2*Skerr2 + Skerr3*Skerr3)
Skerrhat3 = S3/sp.sqrt(Skerr1*Skerr1 + Skerr2*Skerr2 + Skerr3*Skerr3)

Appending to SEOBNR/Hamiltonian.txt


<a id='HssTerm2coeff'></a>
## Step 5b: $H_{\rm SS}$ Term 2 Coefficient \[Back to [top](#toc)\]
$$\label{HssTerm2coeff}$$

We have

\begin{equation}
    H_{\rm SS}\ {\rm Term\ 2\ coefficient} = \frac{ e^{-3 \tilde{\mu} -\nu} \tilde{J} \omega_{r} }{ 2 \tilde{B} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2} }.
\end{equation}

We define [$e^{\tilde{\mu}}$](#exp2mu), [$e^{\nu}$](#exp2nu), [$\tilde{J}$](#Jtilde), [$\omega_{r}$](#omegar), [$\tilde{B}$](#Btilde), [$Q$](#Q),  and [$\xi^{2}$](#xi) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

HssTerm2coeff = Jtilde*omegar/(exp2mu*expmu*expnu*2*Btilde*(sp.sqrt(Q)+1)*sp.sqrt(Q)*xi2)

<a id='omegar'></a>
### $\omega_{r}$ \[Back to [top](#toc)\]
$$\label{omegar}$$

From BB2010 Equation (5.47b) we have

\begin{equation}
    \omega_{r} = \frac{ - \Lambda_{t}^{\prime} \tilde{\omega}_{\rm fd} + \Lambda_{t} \tilde{\omega}_{\rm fd}^{\prime} }{ \Lambda_{t}^{2} }.
\end{equation}

We define [$\Lambda_{t}^{\prime}$](#Lambdatprm), [$\tilde{\omega}_{\rm fd}$](#omegatilde), [$\tilde{\omega}_{\rm fd}^{\prime}$](#omegatildeprm), and [$\Lambda_{t}$](#Lambdat) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

omegar = (Lambdat*omegatildeprm - Lambdatprm*omegatilde)/(Lambdat*Lambdat)

<a id='Lambdatprm'></a>
### $\Lambda_{t}^{\prime}$ \[Back to [top](#toc)\]
$$\label{Lambdatprm}$$

From the discussion after BB2010 Equation (5.47), we know that the prime notation indicates a derivative with respect to $r$.  Using the definiton of [$\Lambda_{t}$](#Lambdat), we have

\begin{equation}
    \Lambda_{t}^{\prime} = 4 \left( a^{2} + r^{2} \right) r - 2 a^{2} r \Delta_{u} \sin^{2} \theta.
\end{equation}

We define [$a^{2}$](#a_terms), [$\Delta_{u}$](#Deltau), and [$\sin^{2}\theta$](#sin2theta) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

Lambdatprm = 4*(a2 + r2)*r - 2*a2*r*Deltau*sin2theta

<a id='omegatildeprm'></a>
### $\tilde{\omega}_{\rm fd}^{\prime}$ \[Back to [top](#toc)\]
$$\label{omegatildeprm}$$

From the discussion after BB2010 Equation (5.47), we know that the prime notation indicates a derivative with respect to $r$.  Using the definiton of [$\tilde{\omega}_{\rm fd}$](#omegatilde), we have

\begin{equation}
    \tilde{\omega}_{\rm fd}^{\prime} = 2 a M.
\end{equation}

We define [$a$](#a_terms) and [$M$](#mass_terms) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

omegatildeprm = 2*a*M

<a id='HssTerm2'></a>
## Step 5c: $H_{\rm SS}$ Term 2 \[Back to [top](#toc)\]
$$\label{HssTerm2}$$

We have

\begin{equation}
    H_{\rm SS}\ {\rm Term\ 2} = -e^{\tilde{\mu} + \nu} \left( {\bf \hat{p}} \cdot {\bf v} r \right) \left( {\bf \hat{p}} \cdot {\bf \xi} r \right) \left( {\bf S} \cdot {\bf \xi} \right)
\tilde{B} + e^{2 \left( \tilde{\mu} + \nu \right)} \left( {\bf \hat{p}} \cdot {\bf \xi} r \right)^2 \left( {\bf S}
\cdot {\bf v} \right) + e^{2 \tilde{\mu}} \left( 1 + \sqrt{Q} \right) \sqrt{Q} \left( {\bf S} \cdot {\bf v} \right)\xi^2 \tilde{B}^{2} + \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left[ \left( {\bf \hat{p}} \cdot {\bf v} r \right)
\left( {\bf S} \cdot {\bf n}\right) - \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf S} \cdot {\bf v} \right)\right] \xi^{2} \tilde{B}^{2}.
\end{equation}

We define [$e^{\tilde{\mu}}$](#exp2mu), [$e^{\nu}$](#exp2nu), [$\hat{\bf p} \cdot {\bf v} r$](#pdotvr), [$\hat{\bf p} \cdot \boldsymbol{\xi} r$](#pdotxir), [${\bf S} \cdot \boldsymbol{\xi}$](#Sdotxi), [$\tilde{B}$](#Btilde), [${\bf S} \cdot {\bf v}$](#Sdotv), [$Q$](#Q), [$\tilde{J}$](#Jtilde), [$\hat{\bf p} \cdot {\bf n}$](#pdotn), [${\bf S} \cdot {\bf n}$](#Sdotn), and [$\xi^{2}$](#xi) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

HssTerm2 = -expmu*expnu*pdotvr*pdotxir*Sdotxi*Btilde + exp2mu*exp2nu*pdotxir**2*Sdotv
            + exp2mu*(1 + sp.sqrt(Q))*sp.sqrt(Q)*Sdotv*xisq*Btilde**2
            + Jtilde*pdotn*(pdotvr*Sdotn - Jtilde*pdotn*Sdotv)*xisq*Btilde**2

<a id='HssTerm3coeff'></a>
## Step 5d: $H_{\rm SS}$ Term 3 Coefficient \[Back to [top](#toc)\]
$$\label{HssTerm3coeff}$$

We have

\begin{equation}
    H_{\rm SS}\ {\rm Term\ 3\ coefficient} = \frac{ e^{-3 \tilde{\mu} - \nu} \omega_{\cos\theta} }{ 2 \tilde{B} \left( \sqrt{Q} + 1 \right) \sqrt{Q} }.
\end{equation}

We define [$e^{\tilde{\mu}}$](#exp2mu), [$e^{\nu}$](#exp2nu), [$\tilde{J}$](#Jtilde), [$\omega_{\cos\theta}$](#omegacostheta), and [$\tilde{B}$](#Btilde), [$Q$](#Q).

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

HssTerm3coeff = omegacostheta/(exp2mu*expmu*expnu*2*Btilde*(sp.sqrt(Q) + 1)*sp.sqrt(Q))

<a id='HssTerm3'></a>
## Step 5e: $H_{\rm SS}$ Term 3 \[Back to [top](#toc)\]
$$\label{HssTerm3}$$

We have

\begin{equation}
    H_{\rm SS}\ {\rm Term\ 3} = -e^{2 \left( \tilde{\mu} + \nu \right)} \left( \hat{\bf p} \cdot {\bf \xi} r \right)^{2} \left( {\bf S} \cdot {\bf n} \right) + e^{\tilde{\mu} +\nu} \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf \hat{p}} \cdot {\bf \xi} r \right) \left( {\bf S} \cdot {\bf \xi} \right) \tilde{B} + \left[ \left( {\bf S} \cdot {\bf n} \right) \left( {\bf \hat{p}} \cdot {\bf v} r \right)^{2} - \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf S} \cdot {\bf v} \right) \left( {\bf \hat{p}} \cdot {\bf v} r\right) - e^{2 \tilde{\mu}} \left( 1 + \sqrt{Q} \right) \sqrt{Q} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B}^{2}.
\end{equation}

We define [$e^{\tilde{\mu}}$](#exp2mu), [$e^{\nu}$](#exp2nu), [$\hat{\bf p} \cdot \boldsymbol{\xi} r$](#pdotxir), [${\bf S} \cdot {\bf n}$](#Sdotn), [$\tilde{J}$](#Jtilde), [$\hat{\bf p} \cdot {\bf n}$](#pdotn), [${\bf S} \cdot \boldsymbol{\xi}$](#Sdotxi), [$\tilde{B}$](#Btilde), [$\hat{\bf p} \cdot {\bf v} r$](#pdotvr), [${\bf S} \cdot {\bf v}$](#Sdotv), [$Q$](#Q), and [$\xi^{2}$](#xi) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

HssTerm3 = -exp2mu*exp2nu*pdotxir**2*Sdotn + expmu*expnu*Jtilde*pdotn*pdotxir*Sdotxi*Btilde 
            + (Sdotn*pdotvr**2 - Jtilde*pdotn*Sdotv*pdotvr - exp2mu*(1 + sp.sqrt(Q)*sp.sqrt(Q)*Sdotn*xisq)*Btilde**2)

<a id='HsoHsscommon'></a>

# Step 6: Expressions Common to $H_{\rm SO}$ and $H_{\rm SS}$ \[Back to [top](#toc)\]
$$\label{HsoHsscommon}$$

What follows are definitions of many of the expressions in $H_{\rm SO}$ and $H_{\rm SS}$.

<a id='exp2nu'></a>

## $e^{2\nu}$ \[Back to [top](#toc)\]
$$\label{exp2nu}$$

From BB2010 Equation (5.32), we have

\begin{equation}
    e^{2 \nu} = \frac{ \Delta_{t} \Sigma }{ \Lambda_t }.
\end{equation}

We define [$\Delta_{t}$](#Deltat), [$\Sigma$](#Sigma), and [$\Lambda_{t}$](#Lambdat) elsewhere.

In [8]:
%%writefile -a SEOBNR/Hamiltonian.txt

exp2nu = Deltat*Sigma/Lambdat
expnu = sp.sqrt(exp2nu)

Appending to SEOBNR/Hamiltonian.txt


<a id='exp2mu'></a>

## $e^{2\tilde{\mu}}$ \[Back to [top](#toc)\]
$$\label{exp2mu}$$

From BB2010 Equation (5.50), we have

\begin{equation}
    e^{2 \tilde{\mu}} = \Sigma
\end{equation}

and thus

\begin{equation}
    e^{\tilde{\mu}} = \sqrt{ \Sigma }.
\end{equation}


We define [$\Sigma$](#Sigma) elsewhere.

In [9]:
%%writefile -a SEOBNR/Hamiltonian.txt

exp2mu = Sigma
expnu = sp.sqrt(exp2mu)

Appending to SEOBNR/Hamiltonian.txt


<a id='omegacostheta'></a>
### $\omega_{\cos\theta}$ \[Back to [top](#toc)\]
$$\label{omegacostheta}$$

From BB2010 Equation (5.47f), we have

\begin{equation}
    \omega_{\cos\theta} = -\frac{ 2 a^{2} \cos\theta \Delta_{t} \tilde{\omega}_{\rm fd} }{ \Lambda_{t}^{2} }.
\end{equation}

We define [$a$](#a_terms), [$\cos\theta$](#costheta), [$\Delta_{t}$](#Deltat), [$\tilde{\omega}_{\rm fd}$](#omegatilde), and [$\Lambda_{t}$](#Lambdat) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

omegacostheta = -2*a2*costheta*Deltat*omegatilde/(Lambdat**2)

<a id='Hnsradicand'></a>

# Step 7: $H_{\rm NS}$ Radicand \[Back to [top](#toc)\]
$$\label{HsoHsscommon}$$

Recall

\begin{equation}
    H_{\rm NS}\ {\rm radicand} = \mu^{2} + \underbrace{\gamma^{ij} p_{i} p_{j}}_{\rm gammappsum} + {\cal Q}_{4} \left( p \right)
\end{equation}

We define [$\eta$](#mass_terms), [gammappsum](#gammappsum), and [${\cal Q}_{4}$](#Q4) elsewhere.

In [23]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hnsradicand = mu**2 + gammappsum + Q4

Appending to SEOBNR/Hamiltonian.txt


<a id='gammappsum'></a>

### $\gamma^{ij} p_{i} p_{j}$ \[Back to [top](#toc)\]
$$\label{gammappsum}$$

From BB2010 Equation (5.46), we have

\begin{equation}
    \gamma^{ij} = g^{ij} - \frac{ g^{ti} g^{tj} }{ g^{tt} }.
\end{equation}

Combining this result with BB2010 Equations 5.36, we have

\begin{equation}
    \gamma^{r\theta} = \gamma^{r\phi} = \gamma^{\theta r} = \gamma^{\theta\phi} = \gamma^{\phi r} = \gamma^{\phi\theta} = 0
\end{equation}

and

\begin{align}
    \gamma^{rr} &= g^{rr} = \frac{ \Delta_{r} }{ \Sigma } \\
    \gamma^{\theta\theta} &= g^{\theta\theta} = \Sigma^{-1} \\
    \gamma^{\phi\phi} &= \frac{ \Sigma }{ \Lambda_{t} \sin^{2} \theta }.
\end{align}

Therefore

\begin{align}
    \gamma^{ij} p_{i} p_{j} &= \gamma^{rr} p_{r} p_{r} + \gamma^{\theta\theta} p_{\theta} p_{\theta} + \gamma^{\phi\phi} p_{\phi} p_{\phi} \\
        &= \frac{ \Delta_{r} }{ \Sigma } p_{r}^{2} + \Sigma^{-1} p_{\theta}^{2} + \frac{ \Sigma }{ \Lambda_{t} \sin^{2} \theta } p_{\phi}^{2}.
\end{align}

Converting Boyer-Lindquist coordinates to tortoise coordinates <font color='red'>(find reference)</font>, we have

\begin{align}
    p_{r} = \hat{\bf p} \cdot {\bf n} \\
    p_{\theta} = \hat{\bf p} \cdot {\bf v} \frac{ r }{ \sin \theta } \\
    p_{\phi} = \hat{\bf p} \cdot \boldsymbol{\xi} r
\end{align}

Therefore

\begin{equation}
    \gamma^{ij} p_{i} p_{j} = \frac{ \Delta_{r} }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2} + \Sigma^{-1} \left( \hat{\bf p} \cdot {\bf v} \frac{ r }{ \sin \theta } \right)^{2} + \frac{ \Sigma }{ \Lambda_{t} \sin^{2} \theta } \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right)^{2}.
\end{equation}

We define [$\Delta_{r}$](#Deltar), [$\Sigma$](#Sigma), [$\hat{\bf p} \cdot {\bf n}$](#pdotn), [$\hat{\bf p} \cdot {\bf v} r$](#pdotvr), [$\sin^{2} \theta$](#sin2theta), [$\Lambda_{t}$](#Lambdat), and [$\hat{\bf p} \cdot \boldsymbol{\xi} r$](#pdotxir) elsewhere.

In [24]:
%%writefile -a SEOBNR/Hamiltonian.txt

gammappsum = Deltar/Sigma*pdotn**2 + 1/Sigma*pdotvr/sin2theta + Sigma/Lambdat/sin2theta*pdotxir**2

Appending to SEOBNR/Hamiltonian.txt


<a id='Q4'></a>

## ${\cal Q}_{4}$ \[Back to [top](#toc)\]
$$\label{Q4}$$

From [Taracchini et. al. 2012](https://arxiv.org/abs/1202.0790) Equation (15),

\begin{equation}
    {\cal Q}_{4} \propto \frac{ prT^{4} }{ r^{2} } \left( r^{2} + \chi_{\rm Kerr}^{2} \right)^{4}.
\end{equation}

On Line 340 of LALSimIMRSpinEOBHamiltonianPrec.c <font color='red'>(find a real reference for this)</font>, we find

\begin{equation}
    {\cal Q}_{4} = \frac{ prT^{4} }{ r^{2} } z_{3}
\end{equation}

where $z_{3}$ is found in Equation (4.34) of [Damour et. al. 2000](https://arxiv.org/abs/gr-qc/0005034):

\begin{equation}
    z_{3} = 2 \left( 4 - 3 \nu \right) \nu
\end{equation}

where in the notation of BB2010 $\nu = \eta$ <font color='red'>(confirm: there's some funny use of $\nu$ in Taracchini)</font>.  Thus

\begin{equation}
    {\cal Q}_{4} = 2 prT^{4} u^{2} \left( 4 - 3 \eta \right) \eta.
\end{equation}

We define [prT](#prT) and [$\eta$](#eta) elsewhere.

In [25]:
%%writefile -a SEOBNR/Hamiltonian.txt

Q4 = 2*prT**4*u2*(4-3*eta)*eta

Appending to SEOBNR/Hamiltonian.txt


<a id='HdTerms'></a>

# Step 8: The $H_{\rm D}$ Terms \[Back to [top](#toc)\]
$$\label{HdTerms}$$

Recall

\begin{equation}
    H_{\rm D} = H_{\rm D}\ {\rm coeffecient} * H_{\rm D}\ {\rm sum}.
\end{equation}

In this step we break down each of [$H_{\rm D}\ {\rm coefficient}$](#Hdcoeff) and [$H_{\rm D}\ {\rm sum}$](#Hdsum).

<a id='Hdcoeff'></a>

## Step 8a: $H_{\rm D}$ Coefficient \[Back to [top](#toc)\]
$$\label{Hdcoeff}$$

We have

\begin{equation}
    H_{\rm D}\ {\rm coefficient} = -\frac{ \eta }{ 2 r^{3} },
\end{equation}

and to save a few FLOPs we'll write

\begin{equation}
    H_{\rm D}\ {\rm coefficient} = -\frac{ \eta u^{3} }{ 2 }.
\end{equation}

We define [$\eta$](#mass_terms) and [$u$](#radius_terms) elsewhere.

In [26]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hdcoeff = -sp.Rational(1,2)*eta*u3

Appending to SEOBNR/Hamiltonian.txt


<a id='Hdsum'></a>

## Step 8b: $H_{\rm D}$ Sum \[Back to [top](#toc)\]
$$\label{Hdsum}$$

We write

\begin{align}
    H_{\rm D}\ {\rm sum} &= \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j} \\
        &= \underbrace{\delta^{ij} S^{*}_{i} S^{*}_{j}}_{\rm Term\ 1} - \underbrace{3 n^{i} n^{j} S^{*}_{i} S^{*}_{j}}_{\rm Term\ 2}.
\end{align}

In [27]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hdsum = HdsumTerm1 - HdsumTerm2

Appending to SEOBNR/Hamiltonian.txt


### $H_{\rm D}$ Sum Term 1

We have

\begin{equation}
    H_{\rm D}\ {\rm sum\ Term\ 1} = \delta^{ij} S^{*}_{i} S^{*}_{j}
\end{equation}

where $\delta^{ij}$ is the Kronecker delta:

\begin{equation}
    \delta_{ij} = \left\{ \begin{array}{cc}
        0, & i \not= j \\
        1, & i = j. \end{array} \right.
\end{equation}

Thus we have

\begin{equation}
    H_{\rm D}\ {\rm sum\ Term\ 1} = S^{*}_{1} S^{*}_{1} + S^{*}_{2} S^{*}_{2} + S^{*}_{3} S^{*}_{3}
\end{equation}

We define [${\bf S}^{*}$](#Hreal_spin_combos) as a combination of spins.

In [28]:
%%writefile -a SEOBNR/Hamiltonian.txt

HdsumTerm1 = Sstar1**2 + Sstar2**2 + Sstar3**2

Appending to SEOBNR/Hamiltonian.txt


### $H_{\rm D}$ Sum Term 2

We have

\begin{align}
    H_{\rm D}\ {\rm sum\ Term\ 2} &= 3 n^{i} n^{j} S^{*}_{i} S^{*}_{j} \\
        &= 3 \left( {\bf S}^{*} \cdot {\bf n} \right)^{2} \\
\end{align}


We define [${\bf S}^{*} \cdot {\bf n}$](#Sstardotn) elsewhere.

In [29]:
%%writefile -a SEOBNR/Hamiltonian.txt

HdsumTerm2 = 3*Sstardotn**2

Appending to SEOBNR/Hamiltonian.txt


<a id='metpots'></a>

# Step N-5: The Deformed and Rescaled Metric Potentials \[Back to [top](#toc)\]
$$\label{metpots}$$

We collect here terms of the deformed and scaled metric potentils.  See BB2010 Equations (5.30)--(5.34) and (5.48)--(5.51).

<a id='Btilde'></a>

## $\tilde{B}$ \[Back to [top](#toc)\]
$$\label{Btilde}$$

From BB2010 Equation (5.48), we have

\begin{equation}
    \tilde{B} = \sqrt{ \Delta_{t} }.
\end{equation}

We define [$\Delta_{t}$](#Deltat) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

Btilde = sp.sqrt(Deltat)

<a id='Jtilde'></a>
### $\tilde{J}$ \[Back to [top](#toc)\]
$$\label{Jtilde}$$

From BB2010 Equation (5.51) we have

\begin{equation}
    \tilde{J} = \sqrt{ \Delta_{r} }.
\end{equation}

We define [$\Delta_{r}$](#Deltar) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

Jtilde = sp.sqrt(Deltar)

<a id='metpotderivs'></a>

# Step N-4: Derivatives of the Metric Potential \[Back to [top](#toc)\]
$$\label{metpotderivs}$$

We collect here terms dependent on derivatives of the metric potential (see BB2010 Equations (5.47)).

<a id='mur'></a>
### $\mu_{r}$ \[Back to [top](#toc)\]
$$\label{mur}$$

From BB2010 Equation (5.47d) we have

\begin{equation}
    \mu_{r} =  \frac{ r }{ \Sigma } - \frac{ 1 }{ \sqrt{ \Delta_{r} } }.
\end{equation}

We define [$\Sigma$](#Sigma) and [$\Delta_{r}$](#Deltar) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

mur = r/Sigma - 1/sp.sqrt(Deltar)

<a id='mucostheta'></a>
### $\mu_{\cos \theta}$ \[Back to [top](#toc)\]
$$\label{mucostheta}$$

From BB2010 Equation (5.47h) we have

\begin{equation}
    \mu_{\cos \theta} =  \frac{ a^{2} \cos \theta }{ \Sigma }.
\end{equation}

We define [$a$](#a_terms), [$\cos \theta$](#costheta), and [$\Sigma$](#Sigma) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

mucostheta = a2*costheta/Sigma

<a id='Qminus1'></a>

## Qminus1 \[Back to [top](#toc)\]
$$\label{Qminus1}$$

First, we specify Qminus1:

\begin{equation}
    {\rm Qminus1} = Q - 1.
\end{equation}

We define [$Q$](#Q) elsewhere.

In [30]:
%%writefile -a SEOBNR/Hamiltonian.txt

Qminus1 = Q - 1

Appending to SEOBNR/Hamiltonian.txt


<a id='Q'></a>

## Q \[Back to [top](#toc)\]
$$\label{Q}$$

From BB2010 Equation (5.52),

\begin{equation}
    Q = 1 + \underbrace{ \frac{ \Delta_{r} }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2} }_{\rm DrSipn2} + \underbrace{ \frac{ \Sigma }{ \Lambda_t \sin^{2} \theta } }_{\rm Q\ coefficient\ 1} \left( \smash[b]{ \underbrace{ \hat{\bf p} \cdot \boldsymbol{\xi} r }_{\rm pdotxir} } \right)^{2} + \underbrace{ \frac{ 1 }{ \Sigma \sin^{2} \theta } }_{\rm Q\ coefficient\ 2} \left( \smash[b]{ \underbrace{ \hat{\bf p} \cdot {\bf v} r }_{\rm pdotvr} } \right)^{2};
\end{equation}

We group togther $\frac{ \Delta_{r} }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2}$ as [DrSipn2](#DrSipn2), $\frac{ \Sigma }{ \Lambda_t \sin^{2} \theta }$ as [Q coefficient 1](#Qcoeff1), $\hat{\bf p} \cdot \boldsymbol{\xi} r$ as [$pdotxir$](#pdotxir), $\frac{ 1 }{ \Sigma \sin^{2} \theta }$ as [Q coefficient 2](#Qcoeff2), and $\hat{\bf p} \cdot {\bf v} r$ as [pdotvr](#pdotvr).

In [31]:
%%writefile -a SEOBNR/Hamiltonian.txt

Q = 1 + DrSipn2 + Qcoeff1*pdotxir + Qcoeff2*pdotvr*pdotvr

Appending to SEOBNR/Hamiltonian.txt


<a id='DrSipn2'></a>

## DrSipn2 \[Back to [top](#toc)\]
$$\label{DrSipn2}$$

We'll write

\begin{equation}
    \frac{ \Delta_{r} }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2} = \Delta_{r} \Sigma^{-1} \left( \hat{\bf p} \cdot {\bf n} \right)^{2}.
\end{equation}

We define $\Delta_{r}$ as [Deltar](#Deltar), $\Sigma^{-1}$ as [Sigmainv](#Sigmainv), and $\hat{\bf p} \cdot {\bf n}$ as [pdotn](#pdotn) elsewhere.

In [32]:
%%writefile -a SEOBNR/Hamiltonian.txt

DrSipn2 = Deltar*Sigmainv*pdotn*pdotn

Appending to SEOBNR/Hamiltonian.txt


<a id='Qcoeff1'></a>

## Q Coefficient 1 \[Back to [top](#toc)\]
$$\label{Qcoeff1}$$

We have,

\begin{equation}
    Q\ {\rm coefficient\ 1} = \frac{ \Sigma }{ \Lambda_t \sin^{2} \theta }
\end{equation}

We define [$\Sigma$](#Sigma), [$\Lambda_{t}$](#Lambdat), and [$\sin^{2} \theta$](#sin2theta) elsewhere.

In [33]:
%%writefile -a SEOBNR/Hamiltonian.txt

Qcoeff1 = Sigma/Lambdat/sin2theta

Appending to SEOBNR/Hamiltonian.txt


<a id='pdotxir'></a>

## $\hat{\bf p} \cdot \boldsymbol{\xi} r$ \[Back to [top](#toc)\]
$$\label{pdotxir}$$

We have

\begin{equation}
    \hat{\bf p} \cdot \boldsymbol{\xi} r = \hat{p}_{1} \xi_{1} + \hat{p}_{2} \xi_{2} + \hat{p}_{3} \xi_{3}
\end{equation}

We define [$\hat{\bf p}$](#hatp) and [$\boldsymbol{\xi}$](#xi) elsewhere.

In [34]:
%%writefile -a SEOBNR/Hamiltonian.txt

pdotxir = hatp1*xi1 + hatp2*xi2 + hatp3*xi3

Appending to SEOBNR/Hamiltonian.txt


<a id='Qcoeff2'></a>

## Q Coefficient 2 \[Back to [top](#toc)\]
$$\label{Qcoeff2}$$

We have

\begin{equation}
    Q\ {\rm coefficient\ 2} = \frac{ 1 }{ \Sigma \sin^{2} \theta }
\end{equation}

We define [$\Sigma$](#Sigma) and [$\sin^{2} \theta$](#sin2theta) elsewhere.

In [35]:
%%writefile -a SEOBNR/Hamiltonian.txt

Qcoeff2 = Sigmainv/sin2theta

Appending to SEOBNR/Hamiltonian.txt


<a id='pdotvr'></a>

## $\hat{\bf p} \cdot {\bf v} r$ \[Back to [top](#toc)\]
$$\label{pdotvr}$$

We have

\begin{equation}
    \hat{\bf p} \cdot {\bf v} r = \left( \hat{p}_{1} v_{1} + \hat{p}_{2} v_{2} + \hat{p}_{3} v_{3} \right) r
\end{equation}

We define [$\hat{\bf p}$](#hatp) and [${\bf v}$](#v) elsewhere.

In [36]:
%%writefile -a SEOBNR/Hamiltonian.txt

pdotvr = (hatp1*v1 + hatp2*v2 + hatp3*v3)*r

Appending to SEOBNR/Hamiltonian.txt


<a id='v'></a>

## $\hat{\bf v}$ \[Back to [top](#toc)\]
$$\label{v}$$

From BB2010 Equation (3.39), we have

\begin{equation}
    {\bf v} = {\bf n} \times \boldsymbol{\xi}.
\end{equation}

We define [${\bf n}$](#n) and [$\boldsymbol{\xi}$](#xi) elsewhere.

In [37]:
%%writefile -a SEOBNR/Hamiltonian.txt

v1 = n2*xi3 - n3*xi2
v2 = n1*xi3 + n3*xi1
v3 = n1*xi2 - n2*xi1

Appending to SEOBNR/Hamiltonian.txt


<a id='Lambdat'></a>

## $\Lambda_{t}$ \[Back to [top](#toc)\]
$$\label{Lambdat}$$

From BB2010 Equation (5.39),

\begin{equation}
    \Lambda_{t} = \varpi^{4} - a^{2} \Delta_{t} \sin^{2} \theta.
\end{equation}

We define [$\varpi^{2}$](#w2), [$a^{2}$](#a_terms), [$\Delta_{t}$](#Deltat), and [$\sin \theta$](#sintheta) elsewhere.

In [38]:
%%writefile -a SEOBNR/Hamiltonian.txt

Lambdat = w2**2 - a2*Deltat*sin2theta

Appending to SEOBNR/Hamiltonian.txt


<a id='w2'></a>

## $\varpi^{2}$ \[Back to [top](#toc)\]
$$\label{w2}$$

From BB2010 Equation (4.7),

\begin{equation}
    \varpi^{2} = a^{2} + r^{2}.
\end{equation}

We define [$a^{2}$](#a_terms) elsewhere.

In [39]:
%%writefile -a SEOBNR/Hamiltonian.txt

w2 = r2 + a2

Appending to SEOBNR/Hamiltonian.txt


<a id='sin2theta'></a>

## $\sin^{2} \theta$ \[Back to [top](#toc)\]
$$\label{sin2theta}$$

From From Line 250 of LALSimIMRSpinPrecEOBHamiltonianPrec.c, (<font color='red'>find paper reference?</font>),

\begin{equation}
    \sin^{2} \theta = 1 - cos^{2} \theta.
\end{equation}

We define [$\cos \theta$](#costheta) elsewhere.

In [40]:
%%writefile -a SEOBNR/Hamiltonian.txt

sin2theta = 1 - costheta*costheta

Appending to SEOBNR/Hamiltonian.txt


<a id='Sigmainv'></a>

## $\Sigma^{-1}$ \[Back to [top](#toc)\]
$$\label{Sigmainv}$$

We'll write

\begin{equation}
    {\rm Sigmainv} = \frac{ 1 }{ \Sigma }.
\end{equation}

We define [$\Sigma$](#Sigma) elsewhere.

In [41]:
%%writefile -a SEOBNR/Hamiltonian.txt

Sigmainv = 1/Sigma

Appending to SEOBNR/Hamiltonian.txt


<a id='Sigma'></a>

## $\Sigma$ \[Back to [top](#toc)\]
$$\label{Sigma}$$

From BB2010 Equation (4.5), we have

\begin{equation}
    \Sigma = r2 + a2 \cos^{2} \theta.
\end{equation}

We define [$a2$](#a_terms) and [$\cos \theta$](#costheta) elsewhere.

In [42]:
%%writefile -a SEOBNR/Hamiltonian.txt

Sigma = r2 + a2*costheta*costheta

Appending to SEOBNR/Hamiltonian.txt


<a id='costheta'></a>

## $\cos \theta$ \[Back to [top](#toc)\]
$$\label{costheta}$$

From Line 248 of LALSimIMRSpinPrecEOBHamiltonianPrec.c, (<font color='red'>find paper reference?</font>)

\begin{equation}
    \cos \theta = {\bf e}_{3} \cdot {\bf n} = {\bf e}_{3}^{1} n^{1} + {\bf e}_{3}^{2} n^{2} + {\bf e}_{3}^{3} n^{3}.
\end{equation}

We define [${\bf e}_{3}$](#e3) and [${\bf n}$](#n) elsewhere.

In [43]:
%%writefile -a SEOBNR/Hamiltonian.txt

costheta = e31*n1 + e32*n2 + e33*n3

Appending to SEOBNR/Hamiltonian.txt


<a id='e3'></a>

## ${\bf e}_{3}$ \[Back to [top](#toc)\]
$$\label{e3}$$

From Lines 228--230 of LALSimIMRSpinPrecEOBHamiltonianPrec.c, (<font color='red'>find paper reference?</font>)

\begin{equation}
    {\bf e}_{3} = \frac{ {\bf S}_{\rm Kerr} }{ a }.
\end{equation}

We define [${\bf S}_{\rm Kerr}$](#Hreal_spin_combos) and [$a$](#a_terms) elsewhere.  Note that in the LALSuite code, e31 is written as e3_x (and similarly for other coordinates).

In [44]:
%%writefile -a SEOBNR/Hamiltonian.txt

e31 = Skerr1/a
e32 = Skerr2/a
e33 = Skerr3/a

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltar'></a>

## $\Delta_{r}$ \[Back to [top](#toc)\]
$$\label{Deltar}$$

From BB2010 Equation (5.38),

\begin{equation}
    \Delta_{r} = \Delta_{t} D^{-1}(u).
\end{equation}

We denote $\Delta_{t}$ as [Deltat](#Deltat) and $D^{-1}(u)$ as [Dinv](#Dinv).

In [45]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltar = Deltat*Dinv

Appending to SEOBNR/Hamiltonian.txt


<a id='Dinv'></a>

## $D^{-1}$ \[Back to [top](#toc)\]
$$\label{Dinv}$$

From BB2010 Equation (5.383),

\begin{equation}
    D^{-1} = 1 + \log \left[ 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3} \right].
\end{equation}

We define [eta](#mass_terms), [u2](#radius_terms), and [u3](#radius_terms) elsewhere.

In [46]:
%%writefile -a SEOBNR/Hamiltonian.txt

Dinv = 1 + sp.log(1 + 6*eta*u2 + 2*(26 - 3*eta)*eta*u3)

Appending to SEOBNR/Hamiltonian.txt


<a id='pdotn'></a>

## $\hat{\bf p} \cdot {\bf n}$ \[Back to [top](#toc)\]
$$\label{pdotn}$$

We have

\begin{align}
    {\rm pdotn} &= \hat{\bf p} \cdot {\bf n} \\
        &= \hat{p}_{1} n_{1} + \hat{p}_{2} n_{2} + \hat{p}_{3} n_{3}
\end{align}

We define [$\hat{\bf p}$](#hatp) and [${\bf n}$](#n) elsewhere.

In [47]:
%%writefile -a SEOBNR/Hamiltonian.txt

pdotn = hatp1*n1 + hatp2*n2 + hatp3*n3

Appending to SEOBNR/Hamiltonian.txt


<a id='hatp'></a>

## $\hat{\bf p}$ \[Back to [top](#toc)\]
$$\label{hatp}$$

From the discussion after BB2010 Equation (3.41), we have $\hat{\bf p} = {\bf p}/m$ where $m$ is the mass of a nonspinning test particle and ${\bf p}$ is *conjugate* momentum.  From Lines 319--321 of LALSimIMRSpinEOBHamiltonianPrec.c (<font color='red'>find a reference for this conversion?</font>)

\begin{align}
    \hat{\bf p} = {\bf p} + {\rm prT} \left( 1 - \frac{1}{\rm csi1} \right) {\bf n}
\end{align}

We define [prT](#prT), [csi1](#csi1), and [${\bf n}$](#n) elsewhere.

In [48]:
%%writefile -a SEOBNR/Hamiltonian.txt

hatp1 = p1 + prT*(1 - 1/csi1)*n1
hatp1 = p2 + prT*(1 - 1/csi1)*n2
hatp1 = p3 + prT*(1 - 1/csi1)*n3

Appending to SEOBNR/Hamiltonian.txt


<a id='prT'></a>

## prT \[Back to [top](#toc)\]
$$\label{prT}$$

At this point I am following the LALSuite code; the conversion to EOB tortoise coordinates is apparently in [Pan et. al. 2010](https://arxiv.org/abs/0912.3466v2). (<font color='red'>Find a better/more comprehensive source?</font>)  From Line 317 of LALSimIMRSpinEOBHamiltonianPrec.c, we have

\begin{align}
    {\rm prT} = {\rm csi1}\left( p_{1} n_{1} + p_{2} n_{2} + p_{3} n_{3} \right)
\end{align}

We define [csi1](#csi1) and [${\bf n}$](#n) elsewhere.

In [49]:
%%writefile -a SEOBNR/Hamiltonian.txt

PrT = csi1*(p1n1 + p2n2 + p3n3)

Appending to SEOBNR/Hamiltonian.txt


<a id='csi1'></a>

## csi1 \[Back to [top](#toc)\]
$$\label{csi1}$$

At this point I am following the LALSuite code; the conversion to EOB tortoise coordinates is apparently in [Pan et. al. 2010](https://arxiv.org/abs/0912.3466v2). (<font color='red'>Find a better/more comprehensive source?</font>)  From Line 309 of LALSimIMRSpinEOBHamiltonianPrec.c, we have

\begin{equation}
    {\rm csi1} = 1 + \left( 1 - \left\lvert 1 - \tau \right\rvert \right) \left( csi - 1 \right)
\end{equation}

We define [csi](#csi) elsewhere; $\tau$ is a tortoise coordinate ($\tau \in \left\{ 0, 1 ,2 \right\}$).

In [50]:
%%writefile -a SEOBNR/Hamiltonian.txt

csi1 = 1 + (1 - sp.abs(1-tortoise))*(csi - 1)

Appending to SEOBNR/Hamiltonian.txt


<a id='csi'></a>

## csi \[Back to [top](#toc)\]
$$\label{csi}$$

At this point I am following the LALSuite code; the conversion to EOB tortoise coordinates is apparently in [Pan et. al. 2010](https://arxiv.org/abs/0912.3466v2). (<font color='red'>Find a better/more comprehensive source?</font>)  From Line 307 of LALSimIMRSpinEOBHamiltonianPrec.c and Equation (28) of Pan et. al. 2010, we have

\begin{equation}
    {\rm csi} = \frac{ \sqrt{ \Delta_{t} \Delta_{r} } }{ \varpi^{2} }.
\end{equation}

We define [$\Delta_{t}$](#Deltat), [$\Delta_{r}$](#Deltar), and [$\varpi^{2}$](#w2) elsewhere.

In [51]:
%%writefile -a SEOBNR/Hamiltonian.txt

csi = sp.sqrt(Deltar*Deltat)/w2

Appending to SEOBNR/Hamiltonian.txt


<a id='xi'></a>

## $\boldsymbol{\xi}$ \[Back to [top](#toc)\]
$$\label{xi}$$

From BB2010 Equation (3.38), we have

\begin{equation}
    \boldsymbol{\xi} = {\bf e}_{3} \times {\bf n}.
\end{equation}

We define [${\bf e}_{3}$](#e3) and [${\bf n}$](#n) elsewhere.  Note that by construction (from discussion after BB2010 Equation 5.52)

\begin{equation}
    \xi^{2} = \sin^{2} \theta.
\end{equation}

In [52]:
%%writefile -a SEOBNR/Hamiltonian.txt

xi1 = e32*n3 - e33*n2
xi2 = e31*n3 + e33*n1
xi3 = e31*n2 - e32*n1
xisq = sin2theta

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltat'></a>

## $\Delta_{t}$ \[Back to [top](#toc)\]
$$\label{Deltat}$$

From BB2010 Equation (5.71), we have

\begin{equation}
    \Delta_{t} = r^{2} \Delta_{u}.
\end{equation}

We denote $r^{2}$ by [r2](#radius_terms) and $\Delta_{u}(u)$ by [Deltau](#Deltau).

In [53]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltat = r2*Deltau

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltatprm'></a>

## $\Delta_{t}^{\prime}$ \[Back to [top](#toc)\]
$$\label{Deltatprm}$$

From the discussion after BB2010 Equation (5.47), we know that the prime notation indicates a derivative with respect to $r$.  Since $\Delta_{u}$ has no $r$ dependence, we have

\begin{equation}
    \Delta_{t}^{\prime} = 2 r \Delta_{u}.
\end{equation}

We define [$\Delta_{u}$](#Deltau) elsewhere.

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltatprm = 2*r*Deltau

<a id='Deltau'></a>

## $\Delta_{u}$ \[Back to [top](#toc)\]
$$\label{Deltau}$$

From BB2010 Equation (5.73), we have

\begin{equation}
    \Delta_u = \bar{\Delta}_{u} (u) \left[ \smash[b]{\underbrace{ 1 + \eta \Delta_{0} + \eta \log \left( 1 + \Delta_{1} u + \Delta_{2} u^{2} + \Delta_{3} u^{3} + \Delta_{4} u^{4} \right) }_{\Delta_{u}\ {\rm calibration\ term}}} \vphantom{\underbrace{1}_{n}} \right]
\end{equation}

We denote $\bar{\Delta}_{u}$ by [Deltaubar](#Deltaubar) and define [$\Delta_{u}$ calibration term](#Deltaucalib) elsewhere.  <font color='red'>Note that we take the absolute value of $\Delta_{u}$ in the LALSuite code (see LALSimIMRSpinEOBHamiltonianPrec.c line 279.</font>

In [54]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltau = Deltaubar*Deltaucalib

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltaucalib'></a>

## $\Delta_{u}$ Calibration Term \[Back to [top](#toc)\]
$$\label{Deltaucalib}$$

From BB2010 Equation (5.73), we have

\begin{align}
    \Delta_u\ {\rm calibration\ term} &= 1 + \eta \Delta_{0} + \eta \log \left( 1 + \Delta_{1} u + \Delta_{2} u^{2} + \Delta_{3} u^{3} + \Delta_{4} u^{4} \right) \\
        &= 1 + \eta \left[ \Delta_{0} + \log \left( 1 + \Delta_{1} u + \Delta_{2} u^{2} + \Delta_{3} u^{3} + \Delta_{4} u^{4} \right) \right].
\end{align}

We define [$u$](#radius_terms) elsewhere.  Each of $\Delta_{i}$, $i \in \left\{0, 1, 2, 3, 4\right\}$ are defined [here](#calib_coeffs).  <font color='red'>Note that in the LALSuite code (see LALSimIMRSpinEOBHamiltonianPrec.c line 274 there are two more terms; whence do they originate?</font>

In [55]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltaucalib = 1 + eta*(Delta0 + sp.log(1 + Delta1*u + Delta2*u2 + Delta3*u3 + Delta4*u4))

Appending to SEOBNR/Hamiltonian.txt


<a id='calib_coeffs'></a>

## Calibration Coefficients $\Delta_{i}$, $i \in \left\{0, 1, 2, 3, 4\right\}$ \[Back to [top](#toc)\]
$$\label{calib_coeffs}$$

From BB2010 Equations (5.77)--(5.81), we have

\begin{align}
    \Delta_{0} &= K \left( \eta K - 2 \right) \\
    \Delta_{1} &= -2 \left( \eta K - 1 \right) \left( K + \Delta_{0} \right) \\
    \Delta_{2} &= \frac{1}{2} \Delta_{1} \left( -4 \eta K + \Delta_{1} + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \\
    \Delta_{3} &= \frac{1}{3} \left[ -\Delta_{1}^{3} + 3 \left( \eta K - 1 \right) \Delta_{1}^{2} + 3 \Delta_{2} \Delta_{1} - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_{2} + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{1} \right] \\
    \Delta_{4} &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_{1}^{2} - 2 \Delta_{2} \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_{1}^{4} - 8 \left( \eta K - 1 \right) \Delta_{1}^{3} - 12 \Delta_{2} \Delta_{1}^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_{2} + \Delta_{3} \right] \Delta_{1} + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_{2}^{2} - 4 \Delta_{3} \left( \eta K - 1 \right) \right] \right\}
\end{align}

We define [$K$](#K), [$\eta$](#mass_terms), [$a$](#a_terms), and [$M$](#mass_terms), [etaKminus1](#etaKminus1), and [a2M2inv](#a2M2inv) elsewhere.

In [56]:
%%writefile -a SEOBNR/Hamiltonian.txt

Delta0 = K*(eta*K - 2)
Delta1 = -2*etaKminus1*(K + Delta0)
Delta2 = sp.Rational(1,2)*Delta1*(-4*eta*K + Delta1 + 4) - a2M2inv*etaKminus1**2
Delta3 = sp.Rational(1,3)*(-Delta1***3 + 3*(etaKminus1*Delta1**2 + 3*Delta2*Delta1 - 6*etaKminus1*(-eta*K + Delta2 + 1)
                                            - 3*a2M2inv*etaKminus1**2*Delta1))
Delta4 = sp.Rational(1,12)*(6*a2M2inv*(\Delta1**2 - 2*Delta2)*etaKminus1**2 + 3*Delta1**4 - 8*etaKminus1*Delta1**3
                           -12*Delta2*Delta1**2 + 12*(2*etaKminus1*Delta2 + Delta2)*Delta1
                           + 12*(sp.Rational(94,3) - sp.Rational(41,32)*pi**2)*etaKminus1**2
                           + 6*(Delta2**2 - 4*Delta3*etaKminus1))

Appending to SEOBNR/Hamiltonian.txt


<a id='a2M2inv'></a>

## a2M2inv \[Back to [top](#toc)\]
$$\label{a2M2inv}$$

We have

\begin{equation}
    {\rm a2M2inv} = \frac{ a^{2} }{ M^{2} }
\end{equation}

We define [a2](#a_terms) and [$M$](#mass_terms) elsewhere.

In [57]:
%%writefile -a SEOBNR/Hamiltonian.txt

a2M2inv = a2/(M*M)

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltaubar'></a>

## $\bar{\Delta}_{u}$ \[Back to [top](#toc)\]
$$\label{Deltaubar}$$

From BB2010 Equation (5.75), we have

\begin{equation}
    \bar{\Delta}_u = \frac{ a^{2} u^{2} }{ M^{2} } + \frac{ 1 }{ \eta K - 1 } \left( 2 u + \frac{ 1 }{ \eta K - 1 } \right)
\end{equation}

We denote $a^{2}$ by [a2](#a_terms), $u^{2}$ by [u2](#radius_terms), and $\frac{ 1 }{ \eta K - 1 }$ by [etaKminus1inv](#etaKminus1).

In [58]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltaubar = s2*u2/(M*M) + etaKminus1inv*(2u + etaKminus1inv)

Appending to SEOBNR/Hamiltonian.txt


<a id='etaKminus1inv'></a>

## $\frac{ 1 }{ \eta K - 1 }$ \[Back to [top](#toc)\]
$$\label{etaKminus1inv}$$

We write

\begin{equation}
    {\rm etaKminus1inv} = \frac{ 1 }{ \underbrace{\eta K - 1}_{\rm etaKminus1} }
\end{equation}

We define [etaKminus1](#etaKminus1) elsewhere.

In [59]:
%%writefile -a SEOBNR/Hamiltonian.txt

etaKminus1inv = 1/etaKminus1

Appending to SEOBNR/Hamiltonian.txt


<a id='etaKminus1'></a>

## $\eta K - 1$ \[Back to [top](#toc)\]
$$\label{etaKminus1}$$

We write

\begin{equation}
    {\rm etaKminus1} = \eta K - 1
\end{equation}

We define [$\eta$](#mass_terms) and [$K$](#K) elsewhere.

In [60]:
%%writefile -a SEOBNR/Hamiltonian.txt

etaKminus1 = eta*K - 1

Appending to SEOBNR/Hamiltonian.txt


<a id='K'></a>

## $K$ \[Back to [top](#toc)\]
$$\label{K}$$

Following the discussion in the two paragrahs after BB2010 Equation (6.11), we have

\begin{equation}
    K = 1.4467 \left( 1 - 4 \eta \right)^{2} + 4 \left( 1 - 2 \eta \right) \eta.
\end{equation}

We define [$\eta$](#mass_terms) elsewhere.

In [61]:
%%writefile -a SEOBNR/Hamiltonian.txt

K = 1.4467*(1 - 4*eta)**2 + 4*(1 - 2*eta)*eta

Appending to SEOBNR/Hamiltonian.txt


<a id='a_terms'></a>

## $a$ and $a^{2}$ \[Back to [top](#toc)\]
$$\label{a_terms}$$

From BB2010 Equation (4.9), we have

\begin{equation}
    a = \frac{ \left\lvert {\bf S}_{\rm Kerr} \right\rvert }{ M } = \frac{ \sqrt{ {\bf S}_{\rm Kerr}^{1} {\bf S}_{\rm Kerr}^{1} + {\bf S}_{\rm Kerr}^{2} {\bf S}_{\rm Kerr}^{2} + {\bf S}_{\rm Kerr}^{3} {\bf S}_{\rm Kerr}^{3} } }{ M }
\end{equation}

whence

\begin{equation}
    a^{2} = \frac{ {\bf S}_{\rm Kerr}^{1} {\bf S}_{\rm Kerr}^{1} + {\bf S}_{\rm Kerr}^{2} {\bf S}_{\rm Kerr}^{2} + {\bf S}_{\rm Kerr}^{3} {\bf S}_{\rm Kerr}^{3} }{ M^{2} }.
\end{equation}

We define [${\bf S}_{\rm Kerr}$](#Hreal_spin_combos) elsewhere.

In [62]:
%%writefile -a SEOBNR/Hamiltonian.txt

a2 = (Skerr1*Skerr1 + Skerr2*Skerr2 + Skerr3*Skerr3) / (M*M)
a = sp.sqrt(a2)

Appending to SEOBNR/Hamiltonian.txt


<a id='dotproducts'></a>

# Step U: Dot Productions \[Back to [top](#toc)\]
$$\label{dotproducts}$$

We collect there various common dot products found in the computation of [$H_{\rm real}$](#Hreal).

<a id='Sdotxi'></a>
### ${\bf S} \cdot \boldsymbol{\xi}$ \[Back to [top](#toc)\]
$$\label{Sdotxi}$$

We have

\begin{equation}
    {\bf S} \cdot \boldsymbol{\xi} = {\bf S}^{1} \xi^{1} + {\bf S}^{2} \xi^{2} + {\bf S}^{3} \xi^{3}
\end{equation}

We define [$\xi$](#xi) elsewhere.

In [1]:
%%writefile -a SEOBNR/Hamiltonian.txt

Sdotxi = S1*xi1 + S2*xi2 + S3*xi3

Writing SEOBNR/Hamiltonian.txt


IOError: [Errno 2] No such file or directory: 'SEOBNR/Hamiltonian.txt'

<a id='Hreal_spin_combos'></a>

# Step N-3: $H_{\rm real}$ Spin Combinations ${\bf S}^{*}$ and ${\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{Hreal_spin_combos}$$

From BB2010 Equation (5.63):

\begin{equation}
    {\bf S}^{*} = \boldsymbol{\sigma}^{*} + \frac{ 1 }{ c^{2} } \boldsymbol{\Delta}_{\sigma^{*}}.
\end{equation}

We define [$\boldsymbol{\sigma}^{*}$](#Heff_spin_combos) and [$\boldsymbol{\Delta}_{\sigma^{*}}$](#Deltasigmastar) elsewhere.

From BB2010 Equation (5.64):

\begin{equation}
    {\bf S}_{\rm Kerr} = \boldsymbol{\sigma} + \frac{ 1 }{ c^{2} } \boldsymbol{\Delta}_{\sigma}.
\end{equation}

We define [$\boldsymbol{\sigma}$](#Heff_spin_combos) elsewhere.  In BB2010 Equation (5.67), $\boldsymbol{\Delta}_{\sigma} = 0$.  Furthermore, we take $c = 1$.

In [63]:
%%writefile -a SEOBNR/Hamiltonian.txt

Sstar1 = sigmastar1 + Deltasigmastar1
Sstar2 = sigmastar2 + Deltasigmastar2
Sstar3 = sigmastar3 + Deltasigmastar3
Skerr1 = sigma1
Skerr2 = sigma2
Skerr3 = sigma3

Appending to SEOBNR/Hamiltonian.txt


<a id='Heff_spin_combos'></a>

# Step N-2: $H_{\rm eff}$ Spin Combinations $\boldsymbol{\sigma}$ and $\boldsymbol{\sigma}^{*}$ \[Back to [top](#toc)\]
$$\label{Heff_spin_combos}$$

From BB2010 Equation (5.2):

\begin{equation}
    \boldsymbol{\sigma} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation}

From BB2010 Equation (5.3):

\begin{equation}
    \boldsymbol{\sigma}^{*} = \frac{ m_{2} }{ m_{1} } {\bf S}_{1} + \frac{ m_{1} }{ m_{2} }{\bf S}_{2}.
\end{equation}

These are both written completely in terms of intrinsic physical parameters.

In [64]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigma1 = S1x + S2x
sigma2 = S1y + S2y
sigma3 = S3z + S3z
sigmastar1 = m2/m1*S1x + m1/m2*S2x
sigmastar2 = m2/m1*S1y + m1/m2*S2y
sigmastar3 = m2/m1*S1z + m1/m2*S2z

Appending to SEOBNR/Hamiltonian.txt


<a id='mass_terms'></a>

# Step N-1: Mass Terms $\mu$, $M$, and $\eta$ \[Back to [top](#toc)\]
$$\label{mass_terms}$$

Each of the terms $\mu$, $M$, and $\eta$ are combinations of the massess $m_{1}$, $m_{2}$ and are defined in the discussion before BB2010 Equation (5.1):

\begin{align}
    M &= m_{1} + m_{2} \\
    \mu &= \frac{ m_{1} m_{2} }{ M } \\
    \eta &= \frac{ \mu }{ M }.
\end{align}

Written completely in terms of the intrinsic physical parameters,

\begin{align}
    \mu &= \frac{ m_{2} m_{2} }{ m_{1} + m_{2} } \\
    \eta &= \frac{ m_{1} m_{2} }{ \left( m_{1} + m_{2} \right)^{2} }.
\end{align}

In [65]:
%%writefile -a SEOBNR/Hamiltonian.txt

eta = mu/M
mu = m1*m2/M
M = m1 + m2

Appending to SEOBNR/Hamiltonian.txt


<a id='radius_terms'></a>

# Step ?: Radius Terms $u$ \[Back to [top](#toc)\]
$$\label{radius_terms}$$

Each of the terms $u$, $u2$, $u3$, $u4$, and $u5$ are powers of $r^{-1}$:

\begin{align}
    u &= \frac{ 1 }{ r } \\
    u2 &= u^{2} \\
    u3 &= u^{3} \\
    u4 &= u^{4} \\
    u5 &= u^{5}.
\end{align}

To save a few FLOPs, we also collect powers of $r$:

\begin{align}
    r2 &= r^{2} \\
    r3 &= r^{3}.
\end{align}

In [66]:
%%writefile -a SEOBNR/Hamiltonian.txt

u = 1/r
u2 = u*u
u3 = u2*u
u4 = u3*u
u5 = u4*u
r2 = r*r
r3 = r2*r

Appending to SEOBNR/Hamiltonian.txt


<a id='metric'></a>

# Step N: The Metric $g$ of Choice \[Back to [top](#toc)\]
$$\label{metric}$$

In BB2010 Equations (5.36a)--(5.36e), we are given the appropriate choice of metric via

\begin{align}
    g^{tt} &= -\frac{ \Lambda_{t} }{ \Delta_{t} \Sigma } \\
    g^{rr} &= \frac{ \Delta_{r} }{ \Sigma } \\
    g^{\theta\theta} &= \frac{ 1 }{ \Sigma } \\
    g^{\phi\phi} &= \frac{ 1 }{ \Lambda_{t} } \left( -\frac{ \tilde{\omega}_{\rm fd}^{2} }{ \Delta_{t} \Sigma } + \frac{ \Sigma }{ \sin^{2} \theta } \right) \\
    g^{t\phi} &= -\frac{ \tilde{\omega}_{\rm fd} }{ \Delta_{t} \Sigma }
\end{align}

We define [$\Lambda_{t}$](#Lambda), [$\Delta_{t}$](#Deltat), [$\Sigma$](#Sigma), [$\Delta_{r}$](#Deltar), [$\tilde{\omega}_{\rm fd}$](#omegatilde) elsewhere.

In [67]:
%%writefile -a SEOBNR/Hamiltonian.txt

gtt = -Lambda/Deltat/Sigma
grr = Deltar/Sigma
ghh = 1/Sigma
gpp = 1/Lambda*(-omegatilda**2/Deltat/Sigma + Sigma/(sintheta**2))
gtp = -omegatilde/Deltat/Sigma

Appending to SEOBNR/Hamiltonian.txt


<a id='Sstardotn'></a>

### ${\bf S}^{*} \cdot {\bf n}$ \[Back to [top](#toc)\]
$$\label{Sstardotn}$$

We have

\begin{equation}
    {\bf S}^{*} \cdot {\bf n} = {\bf S}^{*}_{1} n_{1} + {\bf S}^{*}_{2} n_{2} + {\bf S}^{*}_{3} n_{3}
\end{equation}

with [${\bf S}^{*}$](#Hreal_spin_combos) and [${\bf n}$](#n) defined later.

In [68]:
%%writefile -a SEOBNR/Hamiltonian.txt

Sstardotn = Sstar1*n1 + Sstar2*n2 + Sstar3*n3

Appending to SEOBNR/Hamiltonian.txt


<a id='n'></a>

## ${\bf n}$ \[Back to [top](#toc)\]
$$\label{n}$$

From BB2010 Equation (3.37), we have

\begin{equation}
    {\bf n} = \frac{\bf x }{ r }
\end{equation}

where $r = \left\lvert {\bf x} \right\rvert$ and ${\bf x} = (x, y, z)$.

In [69]:
%%writefile -a SEOBNR/Hamiltonian.txt

n1 = x/r
n2 = y/r
n3 = z/r

Appending to SEOBNR/Hamiltonian.txt


<a id='Deltasigmastar'></a>

### $\boldsymbol{\Delta}_{\sigma^{*}}$ \[Back to [top](#toc)\]
$$\label{Deltasigmastar}$$

We can write $\boldsymbol{\Delta}_{\sigma^{*}}$ as

\begin{equation}
    \boldsymbol{\Delta}_{\sigma^{*}} = \boldsymbol{\sigma}^{*} \left( \boldsymbol{\sigma}^{*}\ {\rm coefficient} \right) + \boldsymbol{\sigma} \left( \boldsymbol{\sigma}\ {\rm coefficient} \right)
\end{equation}

For further dissection, see [$\boldsymbol{\sigma}^{*}$](#Heff_spin_combos), [$\boldsymbol{\sigma}^{*}\ {\rm coefficient}$](#sigmastarcoeff) and [$\boldsymbol{\sigma}$](#Heff_spin_combos), and [$\boldsymbol{\sigma}\ {\rm coefficient}$](#sigmacoeff).

In [70]:
%%writefile -a SEOBNR/Hamiltonian.txt

Deltasigmastar1 = sigmastar1*sigmastarcoeff + sigma1*sigmacoeff
Deltasigmastar2 = sigmastar2*sigmastarcoeff + sigma2*sigmacoeff
Deltasigmastar3 = sigmastar3*sigmastarcoeff + sigma3*sigmacoeff

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmastarcoeff'></a>

### $\boldsymbol{\sigma}^{*}$ coefficient \[Back to [top](#toc)\]
$$\label{sigmastarcoeff}$$

We will break down both $\boldsymbol{\sigma}^{*}\ {\rm coefficient}$ into three terms:

\begin{equation}
    \boldsymbol{\sigma}^{*}\ {\rm coefficient} = \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 1} + \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 2} + \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 3}
\end{equation}

See [$\boldsymbol{\sigma}^{*}$ coefficient Term 1](#sigmastarcoeffTerm1), [$\boldsymbol{\sigma}^{*}$ coefficient Term 2](#sigmastarcoeffTerm2), and [$\boldsymbol{\sigma}^{*}$ coefficient Term 3](#sigmastarcoeffTerm3).

In [71]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmastarcoeff = sigmastarcoeffTerm1 + sigmastarcoeffTerm2 + sigmastarcoeffTerm3

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmacoeff'></a>

### $\boldsymbol{\sigma}$ coefficient \[Back to [top](#toc)\]
$$\label{sigmacoeff}$$

We will break down $\boldsymbol{\sigma}\ {\rm coefficient}$ into three terms:

\begin{equation}
    \boldsymbol{\sigma}\ {\rm coefficient} = \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 1} + \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 2} + \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 3}
\end{equation}

See [$\boldsymbol{\sigma}$ coefficient Term 1](#sigmacoeffTerm1), [$\boldsymbol{\sigma}$ coefficient Term 2](#sigmacoeffTerm2), and [$\boldsymbol{\sigma}$ coefficient Term 3](#sigmacoeffTerm3).

In [72]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmacoeff = sigmacoeffTerm1 + sigmacoeffTerm2 + sigmacoeffTerm3

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmastarcoeffTerm1'></a>

### $\boldsymbol{\sigma}^{*}$ Coefficient Term 1 \[Back to [top](#toc)\]
$$\label{sigmastarcoeffTerm1}$$

We build this term from BB2011 Equation (51) with $b_{0} = 0$ (<font color='red'>why?</font>), where what is listed below is the coefficient on $\boldsymbol{\sigma}^{*}$:

\begin{align}
    \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 1} &= \frac{7}{6} \eta \frac{M}{r} + \frac{1}{3} \eta \left( Q - 1 \right) - \frac{5}{2} \eta \frac{ \Delta_r }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \\
        &= \frac{ \eta }{ 12 } \left( 14 \frac{ M }{ r } + 4 \left( Q - 1 \right) - 30 \frac{ \Delta_r }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \right)
\end{align}

We group together the following: $Q-1$ as [Qminus1](#Qminus1) and $\frac{ \Delta_r }{ \Sigma }$ as [$DrSinp2$](#DrSinp2).  We define [$\eta$](#mass_terms) and [$M$](#mass_terms) elsewhere.

In [73]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmastarcoeffTerm1 = eta/12*(14*M/r + 4*Qminus1 - 30*DrSinp2)

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmastarcoeffTerm2'></a>

### $\boldsymbol{\sigma}^{*}$ Coefficient Term 2 \[Back to [top](#toc)\]
$$\label{sigmastarcoeffTerm2}$$

We build this term from BB2011 Equation (52) with all $b_{i} = 0$, $i \in \left\{0, 1, 2, 3\right\}$ (<font color='red'>why?</font>), and just the coefficient on $\boldsymbol{\sigma}^{*}$.  In the LALSuite code this is the variable 'sMultiplier1':

\begin{align}
    \boldsymbol{\sigma}^{*}\ {\rm coefficient\ Term\ 2} &= \frac{1}{36} \left( 353 \eta - 27 \eta^2 \right) \left( \frac{M}{r} \right)^{2} + \frac{5}{3} \left( 3 \eta^2 \right) \frac{ \Delta_{r}^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4} \\
        &+ \frac{1}{72} \left( -23 \eta -3 \eta^{2} \right) \left( Q - 1 \right)^{2} + \frac{1}{36} \left( -103 \eta + 60 \eta^{2} \right) \frac{M}{r} \left( Q - 1 \right) \\
        &+ \frac{1}{12} \left( 16 \eta - 21 \eta^{2} \right) \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \left( Q - 1 \right) + \frac{1}{12} \left( 47 \eta - 54 \eta^{2} \right) \frac{M}{r} \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \\
        &= \frac{ \eta }{ 72 r^{2} } \left[ \left( 706 - 54 \eta \right) M^{2} + 360 \eta r^{2} \frac{ \Delta_{r}^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4} + r^{2} \left( -23 - 3 \eta \right) \left( Q - 1 \right)^{2} + \left( -206 + 120 \eta \right) M r \left( Q - 1 \right) \right. \\
        &+ \left. \left( 96 - 126 \eta \right) r^{2} \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \left( Q - 1 \right) + \left( 282 - 324 \eta \right) M r \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \right] \\
        &= \frac{ \eta }{ 72 r^{2} } \left[ 706 + r \left( -206 M \left( Q - 1 \right) + 282 M \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} + r \left( Q -1 \right) \left( 96 \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} - 23 \left( Q - 1 \right) \right) \right) \right. \\
        &+ \left. \eta \left( 54 M^{2} + r \left( 120 M \left( Q -1 \right) - 324 M \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} + r \left( 360 \frac{ \Delta_{r}^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4} + \left( Q - 1 \right) \left( -126 \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} - 3 \left( Q - 1 \right) \right) \right)\right) \right) \right]
\end{align}

We define [$\eta$](#mass_terms), [$M$](#mass_terms), [$Q - 1$](#Qminus1), and [$\frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2}$](#DrSinp2) elsewhere.

In [74]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmastarcoeffTerm2 = eta/(72*r**2) ( 706 + r*( -206*M*Qminus1 + 282*M*DrSinp2 + r*Qminus1*(96*DrSinp2 - 23*Qminus1))
                                   + eta*(54*M**2 + r*(120*M*Qminus1-324*M*DrSinp2 + r*(360*DrSinp2**2 + Qminus1*(-126*DrSinp2 - 3*Qminus1)))))

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmastarcoeffTerm3'></a>

### $\boldsymbol{\sigma}^{*}$ Coefficient Term 3 \[Back to [top](#toc)\]
$$\label{sigmastarcoeffTerm3}$$

<font color='red'>I have no idea whence this term originates; there's not any helpful reference in the code.  Unless I'm missing something...</font>

In [75]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmastarcoeffTerm3 =

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmacoeffTerm1'></a>

### $\boldsymbol{\sigma}$ Coefficient Term 1 \[Back to [top](#toc)\]
$$\label{sigmacoeffTerm1}$$

We build this term from BB2011 Equation (51) with $a_{0} = 0$ (<font color='red'>why?</font>), where what is listed below is the coefficient on $\boldsymbol{\sigma}$:

\begin{align}
    \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 1} &= -\frac{2}{3} \eta \frac{ M }{ r } + \frac{1}{4} \eta \left( Q - 1 \right) - 3 \eta \frac{ \Delta_r }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \\
        &= \frac{ \eta }{ 12 } \left( -8 \frac{ M }{ r } + 3 \left( Q - 1 \right) - 36 \smash[b]{\underbrace{ \frac{ \Delta_r }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} }_{\rm DrSipn2}} \vphantom{\underbrace{a}_{b}} \right)
\end{align}

We define [$\eta$](#mass_terms), [$M$](#mass_terms), [$Q-1$](#Qminus1), and [DrSipn2](#DrSipn2) elsewhere.

In [76]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmacoeffTerm1 = eta/12*(-8*M/r + 3*Qminus1 - 36*DrSinp2)

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmacoeffTerm2'></a>

### $\boldsymbol{\sigma}$ Coefficient Term 2 \[Back to [top](#toc)\]
$$\label{sigmacoeffTerm2}$$

We build this term from BB2011 Equation (52) with all $a_{i} = 0$, $i \in \left\{0, 1, 2, 3\right\}$ (<font color='red'>why?</font>), and just the coefficient on $\boldsymbol{\sigma}$:

\begin{align}
    \boldsymbol{\sigma}\ {\rm coefficient\ Term\ 2} &= \frac{1}{9} \left( -56 \eta -21 \eta^{2} \right) \left( \frac{ M }{ r } \right)^{2} + \frac{5}{24} \left( 27 \eta^{2} \right) \frac{ \Delta_r^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4} \\
        &+ \frac{1}{144} \left(-45 \eta \right) \left( Q - 1 \right)^{2} + \frac{1}{36} \left( -109 \eta + 51 \eta^{2} \right) \frac{ M }{ r } \left( Q - 1 \right) \\
        &+ \frac{1}{24} \left( 6 \eta - 39\eta^{2} \right) \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \left( Q - 1 \right) + \frac{1}{24} \left( -16 \eta - 147 \eta^{2} \right) \frac{ M }{ r } \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \\
        &= \frac{ \eta }{ 144 r^{2} } \left[ -896 M^{2} + r \left( -436 M \left( Q - 1 \right) - 96 M \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} + r \left( -45 \left( Q - 1 \right)^{2} + 36 \left( Q - 1 \right) \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \right) \right) \right. \\
        &+ \left. \eta \left( -336 M^{2} + r \left( 204 M \left( Q -1 \right) - 882 M \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} + r \left( 810 \frac{ \Delta_{r}^{2} }{ \Sigma^{2} } \left( {\bf n} \cdot \hat{\bf p} \right)^{4}  - 234 \left( Q - 1 \right) \frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2} \right) \right) \right) \right]
\end{align}

We define [$\eta$](#mass_terms), [$M$](#mass_terms), [$Q - 1$](#Qminus1), and [$\frac{ \Delta_{r} }{ \Sigma } \left( {\bf n} \cdot \hat{\bf p} \right)^{2}$](#DrSinp2) elsewhere.

In [77]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmacoeffTerm2 = eta/(144*r**2)*(-896*M**2 + r*(-436*Qminus1 - 96*M*DrSinp2 + r*(-45*Qminus1**2 + 36*Qminus1*DrSinp2))
                                 + eta*(-336*M**2 + r*(204*M*Qminus1 - 882*M*DrSinp2 + r*(810*DrSinp2**2 - 234*Qminus1*DrSinp2))))

Appending to SEOBNR/Hamiltonian.txt


<a id='sigmacoeffTerm3'></a>

### $\boldsymbol{\sigma}$ Coefficient Term 3 \[Back to [top](#toc)\]
$$\label{sigmacoeffTerm3}$$

<font color='red'>I have no idea whence this term originates; there's not any helpful reference in the code.  Unless I'm missing something...</font>

In [78]:
%%writefile -a SEOBNR/Hamiltonian.txt

sigmacoeffTerm3 =

Appending to SEOBNR/Hamiltonian.txt


<a id='betapsum'></a>

### $\beta^{i} p_{i}$ \[Back to [top](#toc)\]
$$\label{betapsum}$$

From BB2010 Equation (5.45), we have
\begin{equation}
    \beta^{i} = \frac{ g^{ti} }{ g^{tt} },
\end{equation}

but from BB2010 Equations (5.36) we see that $g^{tr} = g^{t \theta} = 0$.  Thus only $\beta^{\phi}$ is nonzero.  Combining BB2010 Equations (5.45), (5.36e), and (5.36a), we find

\begin{equation}
    \beta^{\phi} = \frac{ -\frac{ \tilde{\omega}_{\rm fd} }{ \Delta_{t} \Sigma } }{ -\frac{ \Lambda_{t} }{ \Delta_{t} \Sigma } } = \frac{ \tilde{\omega}_{\rm fd} }{ \Lambda_{t} }
\end{equation}

Therefore

\begin{equation}
    \beta^{i} p_{i} = \frac{ \tilde{\omega}_{\rm fd} }{ \Lambda_{t} } p_{\phi}.
\end{equation}

We define [$\tilde{\omega}_{\rm fd}$](#omegatilde), [$\Lambda_{t}$](#Lambdat), and [$p_{\phi}$](#pphi) elsewhere.

In [79]:
%%writefile -a SEOBNR/Hamiltonian.txt

betapsum = omegatilde/Lambdat*pphi

Appending to SEOBNR/Hamiltonian.txt


<a id='omegatilde'></a>

### $\tilde{\omega}_{\rm fd}$ \[Back to [top](#toc)\]
$$\label{omegatilde}$$

From BB2010 Equation (5.40), we have

\begin{equation}
    \tilde{\omega}_{\rm fd} = 2 a M r + \omega_{1}^{\rm fd} \eta \frac{ a M^{3} }{ r } + \omega_{2}^{\rm fd} \eta \frac{ M a^{3} }{ r }.
\end{equation}

but from BB2011, in the discussion after Equations (50) and (51), we set $\omega_{1}^{\rm fd} = \omega_{2}^{\rm fd} = 0$.  Thus

\begin{equation}
    \tilde{\omega}_{\rm fd} = 2 a M r.
\end{equation}

We define [$a$](#a_terms) and [$M$](#mass_terms) elsewhere.

In [80]:
%%writefile -a SEOBNR/Hamiltonian.txt

omegatilde = 2*a*M*r

Appending to SEOBNR/Hamiltonian.txt


<a id='pphi'></a>

### $p_{\phi}$ \[Back to [top](#toc)\]
$$\label{pphi}$$

Converting from Boyer-Lindquist momentum coordinates to tortoise momentum coordinates, <font color='red'>(verify this)</font>,

\begin{equation}
    p_{\phi} = \hat{\bf p} \cdot \boldsymbol{\xi} r.
\end{equation}

We define [$\hat {\bf p} \cdot \boldsymbol{\xi} r$](#pdotxir) and [$M$](#mass_terms) elsewhere.

In [81]:
%%writefile -a SEOBNR/Hamiltonian.txt

pphi = pdotxir

Appending to SEOBNR/Hamiltonian.txt


<a id='alpha'></a>

### $\alpha$ \[Back to [top](#toc)\]
$$\label{alpha}$$

From BB2010 Equation (5.44), we have
\begin{equation}
    \alpha = \frac{ 1 }{ \sqrt{ -g^{tt}} },
\end{equation}

and from BB2010 Equations (5.36a) we have

\begin{equation}
    g^{tt} = -\frac{ \Lambda_{t} }{ \Delta_{t} \Sigma }.
\end{equation}

Therefore

\begin{equation}
    \alpha = \sqrt{ \frac{ \Delta_{t} \Sigma }{ \Lambda_{t} } }.
\end{equation}

We define [$\Delta_{t}$](#Deltat), [$\Sigma$](#Sigma), and [$\Lambda_{t}$](#Lambdat) elsewhere.

In [82]:
%%writefile -a SEOBNR/Hamiltonian.txt

alpha = sp.sqrt(Deltat*Sigma/Lambdat)

Appending to SEOBNR/Hamiltonian.txt


<a id='notes'></a>
# Notes \[Back to [top](#toc)\]
$$\label{notes}$$

1. First, just break down the Hamiltonian as in the paper.  Break up into blocks as appropriate.
2. Use python markdown to write string of input variables:

```python
# Step 0a: declare instrinsic system paramters (masses and spin components)
m1, m2, s1x, s1y, s1z, s2x, s2y, s2z = sp.symbols("m1 m2 s1x s1y s1z s2x s2y s2z",real=True)

```
3. Write terms to files in backwards order!  As in:

    ```python
    #First write is this:
    #%%writefile SEOBNR/Hamiltonian.txt

    ```
   and afterward

    ```python
    #%%writefile -a SEOBNR/Hamiltonian.txt

    ```
   The goal is to split up the terms so that each $\LaTeX$-expression at the bottom of a markdown cell can be easily compared to the "bigstring" expression that is SymPy compatible in a code cell directly below the markdown cell.
4. ${\cal Q}_{4}$ is quartic in $p_{i}$ and is given by Equation (4.34) of [Damour, Jaranowski, and Schaefer](https://arxiv.org/abs/gr-qc/0005034) <font color='red'> (double check this!)</font>

<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [83]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-SEOBNR_Documentation.ipynb to latex
[NbConvertApp] Writing 124767 bytes to Tutorial-SEOBNR_Documentation.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
